# Predicting energies efficiently and on mixed-sites

In [33]:
import xgboost as xgb
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from xgboost import XGBRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import time

In [3]:
#Load H binding energy prediction model
H_DFT_model = xgb.Booster({'nthread': 8})
H_DFT_model.load_model("../models/"+"H_DFT.model")

#Load COOH binding energy prediction model
COOH_DFT_model = xgb.Booster({'nthread': 8})
COOH_DFT_model.load_model("../models/"+"COOH_DFT.model")

models = {"H": H_DFT_model, "COOH": COOH_DFT_model}

### Compare time to make predictions of COOH and H on a 100 by 100 surface with the new way to predict energies

#### Old method:

In [34]:
dim_x, dim_y, split = 100, 100, [1, 1, 1, 1, 1]

# Note the stochiometry - used for plotting
stochiometry = dict(zip(metals, np.array(split)/np.sum(split)))
# Make the surface (empty surface)
surface = initialize_surface(dim_x, dim_y, metals, split)

# Predict energies on all sites for both adsorbates
surface = precompute_binding_energies_TQDM(surface, dim_x, dim_y, models, predict_G)

Predicting all ΔG:   0%|          | 0/10000 [00:00<?, ?it/s]

#### New method:

In [66]:
dim_x, dim_y, split = 100, 100, [1, 1, 1, 1, 1]

# Note the stochiometry - used for plotting
stochiometry = dict(zip(metals, np.array(split)/np.sum(split)))
# Make the surface (empty surface)
surface = initialize_surface(dim_x, dim_y, metals, split)

# Predict energies on all sites for both adsorbates
surface = precompute_binding_energies_SPEED(surface, dim_x, dim_y, models)

#### Functions:

In [65]:
def initialize_surface(dim_x, dim_y, metals, split): #Is still random - could be used with a seed in the name of reproduceability
    dim_z = 3
    
    surf_atoms = create_surface(dim_x, dim_y, metals, split)
    
    # Binding energies
    surf_COOH_G = np.reshape([np.nan]*dim_x*dim_y, (dim_x, dim_y))# On-top sites
    surf_H_G    = np.reshape([np.nan]*dim_x*dim_y, (dim_x, dim_y))# Hollow sites
    
    surf = {"atoms": surf_atoms,\
            "COOH_G": surf_COOH_G, "H_G": surf_H_G}
    return surf

def create_surface(dim_x, dim_y, metals, split):
    dim_z = 3
    num_atoms = dim_x*dim_y*dim_z
    if np.sum(split) != 1.0:
        # This split is not weighted properly, I'll fix it
        split = split / np.sum(split)
    if split == "Even":
        proba = [1.0 / len(metals) for n in range(len(metals))] 
        surface = np.random.choice(metals, num_atoms, p=proba)
    else:
        surface = np.random.choice(metals, num_atoms, p=split)
    surface = np.reshape(surface, (dim_x, dim_y, dim_z)) #Reshape list to the
    return surface

def precompute_binding_energies_TQDM(surface, dim_x, dim_y, models, predict_G_function): #TJEK I think this function can go faster if I make all the data first appended to a list, then to a PD and then 
    for x, y in tqdm([(x, y) for x in range(dim_x) for y in range(dim_y)], desc = r"Predicting all ΔG", leave = False): # I could randomise this, so I go through all sites in a random order
        
        ads = "H"
        surface["H_G"][x][y] = predict_G_function(surface["atoms"], x, y, ads, models) ## A new function that wraps/uses the XGBoost model
        
        ads = "COOH"
        surface["COOH_G"][x][y] = predict_G_function(surface["atoms"], x, y, ads, models) ## A new function that wraps/uses the XGBoost model

    return surface

def precompute_binding_energies_SPEED(surface, dim_x, dim_y, models):
    H_features    = []
    COOH_features = []
    #index_pairs   = []

    # Make features for each site:
    for x, y in [(x, y) for x in range(dim_x) for y in range(dim_y)]:#, desc = r"Making all feature vectors", leave = True): # I could randomise this, so I go through all sites in a random order
        # Append the features
        H_features.append([hollow_site_vector(surface["atoms"], x, y)])
        COOH_features.append([on_top_site_vector(surface["atoms"], x, y)])
        #index_pairs.append([str(x)+","+str(y)])

    # Remove the uneccesary singleton dimension
    H_features = np.squeeze(H_features)
    COOH_features = np.squeeze(COOH_features)

    # Make the features into a big dataframe
    H_features_df    = pd.DataFrame(H_features   , columns = [f"feature{n}" for n in range(55)])
    COOH_features_df = pd.DataFrame(COOH_features, columns = [f"feature{n}" for n in range(20)])

    # Turn them into DMatrix
    H_features_DM    = pandas_to_DMatrix(H_features_df)
    COOH_features_DM = pandas_to_DMatrix(COOH_features_df)

    # Predict energies in one long straight line
    H_G    = models["H"].predict(H_features_DM)
    COOH_G = models["COOH"].predict(COOH_features_DM)

    # Make them into a nice matrix shape - in a minute
    H_G    = np.reshape(H_G   , (dim_x, dim_y))
    COOH_G = np.reshape(COOH_G, (dim_x, dim_y))
    #index_pairs = np.reshape(index_pairs, (dim_x, dim_y))

    # Attach the energies to the matrices in the surface dictionary
    surface["H_G"]    = H_G
    surface["COOH_G"] = COOH_G
    return surface

def predict_G(surface, site_x, site_y, adsorbate, models):
    if adsorbate == "H":
        vector_df = pd.DataFrame([hollow_site_vector(surface, site_x, site_y)], columns = [f"feature{n}" for n in range(55)])
        vector_DM = pandas_to_DMatrix(vector_df)
        G = models["H"].predict(vector_DM)[0]
        return G
    
    if adsorbate == "COOH":
        vector_df = pd.DataFrame([on_top_site_vector(surface, site_x, site_y)], columns = [f"feature{n}" for n in range(20)])
        vector_DM = pandas_to_DMatrix(vector_df)
        G = models["COOH"].predict(vector_DM)[0]
        return G
    
def on_top_site_vector(surface, site_x, site_y): # I should have done modulo to dim_x and dim_y
    dim_x, dim_y = np.shape(surface)[0], np.shape(surface)[1]
    site1 = [surface[site_x, site_y, 0]]# Make a one-hot encoded vector of the very site here! Add at the beginning 
    site1_count = [site1.count(metals[n]) for n in range(len(metals))]
    
    top6 = [surface[site_x % dim_x, (site_y-1) % dim_y, 0], surface[site_x % dim_x, (site_y+1) % dim_y, 0], surface[(site_x-1) % dim_x, site_y % dim_y, 0], surface[(site_x+1) % dim_x, site_y % dim_y, 0], surface[(site_x-1) % dim_x, (site_y+1) % dim_y, 0], surface[(site_x+1) % dim_x, (site_y-1) % dim_y, 0]]
    top6_count = [top6.count(metals[n]) for n in range(len(metals))]
    
    mid3 = [surface[(site_x-1) % dim_x, (site_y-1) % dim_y,1], surface[site_x % dim_x, (site_y-1) % dim_y,1], surface[(site_x-1) % dim_x, site_y % dim_y,1]]
    mid3_count = [mid3.count(metals[n]) for n in range(len(metals))]
    
    bot3 = [surface[(site_x-1) % dim_x, (site_y-1) % dim_y, 2], surface[(site_x-1) % dim_x, (site_y+1) % dim_y, 2], surface[(site_x+1) % dim_x, (site_y-1) % dim_y, 2]]
    bot3_count = [bot3.count(metals[n]) for n in range(len(metals))]
    
    return site1_count + top6_count + mid3_count + bot3_count

metals = ['Ag', 'Au', 'Cu', 'Pd', 'Pt']
three_metals_combinations = [] #List of possible combinations of the three
# Der skal være 35, ikke 125

for a in metals:
    for b in metals:
        for c in metals:
            three_metals_combinations.append(''.join(sorted([a, b, c])))
            
# Remove duplicates
three_metals_combinations = list(dict.fromkeys(three_metals_combinations)) # Let's encode it in a better way later

def hollow_site_vector(surface, site_x, site_y):
    
    # First encode the 3 neighbours
    blues = [surface[(site_x+1) % dim_x, site_y, 0], surface[site_x, (site_y+1) % dim_y, 0], surface[(site_x+1) % dim_x, (site_y+1) % dim_y, 0]]
    blues = "".join(sorted(blues))
    idx = three_metals_combinations.index(blues)
    blues = 35*[0]
    blues[idx] = 1
    
    # Then the next neighbours (green)
    greens = [surface[(site_x+2) % dim_x, site_y, 0], surface[site_x, (site_y+2) % dim_y, 0], surface[site_x, site_y, 0]]
    greens_count = [greens.count(metals[n]) for n in range(len(metals))]
    
    # Then the next neighbours (brown) # Kunne gøres smartere med list comprehension og to lister med +- zipped
    browns = [surface[(site_x + a) % dim_x, (site_y + b) % dim_y, c] for a, b, c in zip([1, 2, 2, 1, -1, -1], [2, 1, -1, -1, 1, 2], [0, 0, 0, 0, 0, 0])]
    browns_count = [browns.count(metals[n]) for n in range(len(metals))]
    
    # Then the three downstairs neighbours
    yellows = [surface[(site_x + a) % dim_x, (site_y + b) % dim_y, c] for a, b, c in zip([0, 1, 0], [0, 0, 1], [1, 1, 1])]
    yellows_count = [yellows.count(metals[n]) for n in range(len(metals))]
    
    # Then the purples downstairs
    purples = [surface[(site_x + a) % dim_x, (site_y + b) % dim_y, c] for a, b, c in zip([1, -1, 1], [-1, 1, 1], [1, 1, 1])]
    purples_count = [purples.count(metals[n]) for n in range(len(metals))]
    
    return blues + greens_count + browns_count + yellows_count + purples_count

def pandas_to_DMatrix(df):#, label):
    label = pd.DataFrame(np.random.randint(2, size=len(df)))
    DMatrix = xgb.DMatrix(df)#, label=label)
    return DMatrix